In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
import numpy as np
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer ## HAndle Missing Values
from sklearn.preprocessing import StandardScaler ## Feature Scaling
from sklearn.preprocessing import OneHotEncoder ## categorical to numerical
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
import xgboost as xb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from imblearn.combine import SMOTETomek
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedGroupKFold
import sweetviz as sv 
pd.set_option('display.max_columns',None)
plt.rcParams['figure.figsize']=(15,10)
from sklearn.feature_selection import mutual_info_classif  ## good for only classification problem 
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile
from sklearn.preprocessing import OrdinalEncoder
from imblearn.over_sampling import RandomOverSampler

In [2]:
df=pd.read_csv('final_loan.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   rate_of_interest      112231 non-null  float64
 1   Interest_rate_spread  112031 non-null  float64
 2   Upfront_charges       109028 non-null  float64
 3   property_value        133572 non-null  float64
 4   credit_type           148670 non-null  object 
 5   LTV                   133572 non-null  float64
 6   dtir1                 124549 non-null  float64
 7   Status                148670 non-null  int64  
dtypes: float64(6), int64(1), object(1)
memory usage: 9.1+ MB


In [4]:
df.duplicated().sum()

16963

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
df.shape

(131707, 8)

In [7]:
df.isnull().sum()

rate_of_interest        19484
Interest_rate_spread    19679
Upfront_charges         22682
property_value              3
credit_type                 0
LTV                         3
dtir1                    8765
Status                      0
dtype: int64

In [10]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [8]:
feature_witn_na_num=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtype!='O']

In [9]:
feature_witn_na_col=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtype=='O']

In [12]:
feature_witn_na_col

[]

In [13]:
for feature in df[feature_witn_na_num]:
    df[feature]=df[feature].fillna(df[feature].median())

In [14]:
df.isnull().sum()

rate_of_interest        0
Interest_rate_spread    0
Upfront_charges         0
property_value          0
credit_type             0
LTV                     0
dtir1                   0
Status                  0
dtype: int64

In [15]:
categorical_column=[feature for feature in df.columns if df[feature].dtype=='O']

In [16]:
for feature in df[categorical_column]:
    print(df[feature].value_counts())
    print('********************')

credit_type
CIB     47487
CRIF    43300
EXP     40722
EQUI      198
Name: count, dtype: int64
********************


In [17]:
def remove_outliers(df,cols):
    for feature in cols:
        Q1=df[feature].quantile(0.25)
        Q3=df[feature].quantile(0.75)
        IQR=Q3-Q1
    
        lower_bound=Q1-1.5*IQR
        upper_bound=Q3+1.5*IQR
        
        df.loc[df[feature] > upper_bound,feature]=upper_bound
        df.loc[df[feature] < lower_bound,feature]=lower_bound

In [19]:
nuemrical_cols=[feature for feature in df.columns if df[feature].dtype!='O']

In [21]:
nuemrical_cols.remove("Status")

In [22]:
remove_outliers(df,nuemrical_cols)

In [23]:
for feature in categorical_column:
    labels_ordered=df.groupby([feature])['Status'].count().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    df[feature]=df[feature].map(labels_ordered)

In [24]:
df

,rate_of_interest,Interest_rate_spread,Upfront_charges,property_value,credit_type,LTV,dtir1,Status
0,3.990,0.3904,2596.56,118000.0,1,98.728814,45.0,1
1,3.990,0.3904,2596.56,418000.0,0,75.088028,39.0,1
2,4.560,0.2000,595.00,508000.0,1,80.019685,46.0,0
3,4.250,0.6810,2596.56,658000.0,1,69.376900,42.0,0
4,4.000,0.3042,0.00,758000.0,2,91.886544,39.0,0
...,...,...,...,...,...,...,...,...
148665,3.125,0.2571,9126.50,608000.0,3,71.792763,48.0,0
148666,5.190,0.8544,0.00,788000.0,3,74.428934,15.0,0
148667,3.125,0.0816,1226.64,728000.0,3,61.332418,49.0,0
148668,3.500,0.5824,4323.33,278000.0,1,70.683453,29.0,0


In [25]:
X_train,X_test,y_train,y_test=train_test_split(df.drop('Status',axis=1),df[['Status']],random_state=42,test_size=0.3)

In [27]:
model=[LogisticRegression(),DecisionTreeClassifier(),GaussianNB(),AdaBoostClassifier()]

In [28]:
def models_feat(X_train,X_test,y_train,y_test,models):
    for model in models:
        model.fit(X_train,y_train)
        y_pred=model.predict(X_test)
        Y_pred=model.predict(X_train)
        print(model,'Training',accuracy_score(y_train,Y_pred))
        print(model,'Testing',accuracy_score(y_test,y_pred))
        print(classification_report(y_test,y_pred))
        print(confusion_matrix(y_test,y_pred))
        print('******************')

In [29]:
models_feat(X_train,X_test,y_train,y_test,model)

LogisticRegression() Training 0.850944746946656
LogisticRegression() Testing 0.849745653329284
              precision    recall  f1-score   support

           0       0.85      1.00      0.92     33576
           1       0.00      0.00      0.00      5937

    accuracy                           0.85     39513
   macro avg       0.42      0.50      0.46     39513
weighted avg       0.72      0.85      0.78     39513

[[33576     0]
 [ 5937     0]]
******************
DecisionTreeClassifier() Training 1.0
DecisionTreeClassifier() Testing 0.9999493837471212
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     33576
           1       1.00      1.00      1.00      5937

    accuracy                           1.00     39513
   macro avg       1.00      1.00      1.00     39513
weighted avg       1.00      1.00      1.00     39513

[[33574     2]
 [    0  5937]]
******************
GaussianNB() Training 0.9245720979673298
GaussianNB() Testing